In [1]:
import os
import sys

import numpy as np

sys.path.append(os.path.join('/home/hassaku/pfn-prediction/src/'))

In [2]:
from modules.dataloader import DataLoader
from modules.network import Network

In [3]:
dataloader = DataLoader('../data/raw/hands/hand_position.json')
dataloader.load_data()
train, valid, test = dataloader.split()
net = Network()

In [4]:
from chainer import optimizers
from chainer.optimizer_hooks import WeightDecay

optimizer = optimizers.SGD(lr=0.001)  # 学習率を 0.01 に設定
optimizer.setup(net)

for param in net.params():
    if param.name != 'b':  # バイアス以外だったら
        param.update_rule.add_hook(WeightDecay(0.0001))  # 重み減衰を適用

In [5]:
from chainer.iterators import SerialIterator
train_iter = SerialIterator(train, batch_size=4, repeat=True, shuffle=True)

In [7]:
'''
config
'''
# gpu_id = 0  # 使用する GPU 番号
n_batch = 2  # バッチサイズ
n_epoch = 5  # エポック数

# ネットワークを GPU メモリ上に転送
# net.to_gpu(gpu_id)

In [8]:
import chainer
import chainer.functions as F
from chainer.dataset import concat_examples

count = 0
# ログの保存用
results_train = {
    'loss': [],
    'accuracy': []
}
results_valid = {
    'loss': [],
    'accuracy': []
}
for epoch in range(n_epoch):
    while True:
        # ミニバッチの取得
        train_batch = train_iter.next()
        # x と t に分割
        # データを GPU に転送するために、concat_examples に gpu_id を渡す
        x_train, t_train = concat_examples(train_batch) #, gpu_id)

        # 予測値と目的関数の計算
        y_train = net(x_train)
        loss_train = F.mean_squared_error(y_train, t_train.astype(np.float32))
        acc_train = F.binary_accuracy(y_train, t_train)

        # 勾配の初期化と勾配の計算
        net.cleargrads()
        loss_train.backward()

        # パラメータの更新
        optimizer.update()

        # カウントアップ
        count += 1

        # 1エポック終えたら、valid データで評価する
        if train_iter.is_new_epoch:

            # 検証用データに対する結果の確認
            with chainer.using_config('train', False), chainer.using_config('enable_backprop', False):
                x_valid, t_valid = chainer.dataset.concat_examples(valid) #, gpu_id)
                y_valid = net(x_valid)
                loss_valid = F.mean_squared_error(y_valid, t_valid.astype(np.float32))
                acc_valid = F.binary_accuracy(y_valid, t_valid)

            # 注意：GPU で計算した結果はGPU上に存在するため、CPU上に転送します
#             loss_train.to_cpu()
#             loss_valid.to_cpu()
#             acc_train.to_cpu()
#             acc_valid.to_cpu()

            # 結果の表示
            print('epoch: {}, iteration: {}, loss (train): {:.4f}, loss (valid): {:.4f}'
                  'acc (train): {:.4f}, acc (valid): {:.4f}'.format(
                epoch, count, loss_train.array.mean(), loss_valid.array.mean(),
                  acc_train.array.mean(), acc_valid.array.mean()))

            # 可視化用に保存
            results_train['loss'] .append(loss_train.array)
            results_train['accuracy'] .append(acc_train.array)
            results_valid['loss'].append(loss_valid.array)
            results_valid['accuracy'].append(acc_valid.array)
            
            break

epoch: 0, iteration: 2, loss (train): 0.1908, loss (valid): 0.1669acc (train): 0.0003, acc (valid): 0.0003
epoch: 1, iteration: 4, loss (train): 0.1031, loss (valid): 0.1248acc (train): 0.0003, acc (valid): 0.0003
epoch: 2, iteration: 6, loss (train): 0.0891, loss (valid): 0.0985acc (train): 0.0003, acc (valid): 0.0003
epoch: 3, iteration: 8, loss (train): 0.0728, loss (valid): 0.0801acc (train): 0.0003, acc (valid): 0.0003
epoch: 4, iteration: 10, loss (train): 0.0636, loss (valid): 0.0666acc (train): 0.0003, acc (valid): 0.0003
